In [ ]:
import geopandas
import rasterio
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from utils import read_raster

%load_ext autoreload
%autoreload 2

In [ ]:
DATA_PATH = Path("data/")
CRS = "EPSG:4326"

### Déplacement des zones propices 

In [ ]:
areas_2011_2040 = geopandas.read_file(DATA_PATH / "rasters" / "arcp8510000532011-2040.shp")
areas_2011_2040.plot()

In [ ]:
areas_2041_2070 = geopandas.read_file(DATA_PATH / "rasters" / "arcp8510000532041-2070.shp")
areas_2041_2070.plot()

In [ ]:
areas_2071_2100 = geopandas.read_file(DATA_PATH / "rasters" / "arcp8510000532071-2100.shp")
areas_2071_2100.plot()

In [ ]:
assert areas_2011_2040.crs == areas_2041_2070.crs
assert areas_2011_2040.crs == areas_2071_2100.crs 

print(crs)

### Boundary files

In [ ]:
boundaries = geopandas.read_file(DATA_PATH / "boundaries" / "lcar000b21a_e.shp")
boundaries = boundaries.to_crs(CRS)
boundaries.plot()

In [ ]:
def _get_area(df):
    # Reproject in a cartesian system to compute area (squared km)
    area = df.geometry.to_crs('EPSG:3857').geometry.area / 10 ** 6
    
    return area

boundaries["total_area"] = _get_area(boundaries)

In [ ]:
boundaries.head().explore()

### Spatial join boundaries and areas 2011-2041

In [ ]:
intersection = boundaries.head().overlay(areas_2011_2040, how="intersection")

In [ ]:
intersection.plot(column="DN")

In [ ]:
intersection.head()

### Calculate areas (squared km) of migration zones

In [ ]:
intersection["intersection_area"] = _get_area(intersection)
intersection.head()

In [ ]:
intersection["proportion"] = intersection["intersection_area"] / intersection["total_area"]
intersection.head()

In [ ]:
intersection.explore()

### Load current proportion maps

In [ ]:
with rasterio.open(DATA_PATH / "cartography" / "ACESAC_prop_250m_final.tif") as src:
    affine = src.transform
    current_prop = src.read(1)
    
#     no_data_val = src.nodatavals[0]
#     current_prop[current_prop == no_data_val] = np.nan

In [ ]:
plt.imshow(current_prop, cmap='pink')
plt.show()

In [ ]:
affine

In [ ]:
from rasterstats import zonal_stats
import pandas as pd

gdf = boundaries.head()
# gdf['mean'] = pd.DataFrame(
zonal_stats(
    vectors=gdf['geometry'], 
    raster=current_prop, 
    affine=affine, 
    stats='mean'
)
# )['mean']

In [ ]:
gdf